In [1]:
import os
import glob
import json
import random
from datastructures.rationals import *
from datastructures.edgebst import *
from datastructures.dcel import *
from visualize.render import *
from algorithms.monotonize import *
from algorithms.triangulate import *
from algorithms.merge import *

instances_dir = "instances/2ima15"
instance_files = glob.glob(instances_dir + "/*.instance.json")
# Range of seeds to be used when using random_permutation
seeds = range(1)

def compute_convex_cover(dcel, times_rotated, permutation):
    dcel.rotate_right(times_rotated)
    monotonize_polygon(dcel)
    triangulate_monotone(dcel, triangulate_convex_faces=False)
    hertel_mehlhorn(dcel, permutation)
    bruteforce_merge_indirect_neighbours(dcel)
    dcel.rotate_left(times_rotated)
    return len(dcel.interior_faces())

def random_permutation(diagonals, seed):
    # Randomly shuffles diagonals
    random.seed(seed)
    random.shuffle(diagonals)

def sort_on_yx(diagonals):
    # Sorts diagonals on lexicographically on their lower endpoint, first y then x
    diagonals.sort(key=lambda h: (h.origin.y, h.origin.x))

def sort_on_face(diagonals):
    # Sorts diagonals on their incident faces, the faces are lexicographically ordered on the origin of their outer component
    diagonals.sort(key=lambda h: (h.incident_face.outer_component.origin.x, h.incident_face.outer_component.origin.y))

total_score = 0
for instance_file in instance_files:
    instance_name = os.path.basename(instance_file)
    result_file = instance_file.replace("instance.json", "result.json")

    input = open(instance_file, "r")
    poly = json.load(input)
    input.close()
    
    min_result = float("inf")
    opt_rotation = None
    opt_solution = None
    opt_seed = None
    for seed in seeds:
        for times_rotated in range(4):
            dcel = DCEL(poly["outer_boundary"], poly["holes"])
            result = compute_convex_cover(dcel, times_rotated, permutation=sort_on_yx) # Permutation here can be changed to test different results
            if result < min_result:
                min_result = result
                opt_rotation = times_rotated
                opt_solution = dcel.format_solution()
                opt_seed = seed

    print(f"{instance_name} (rotated {opt_rotation} times, seed {opt_seed}) - Faces: {min_result}")
    total_score += min_result
    export = opt_solution

    export["instance"] = poly["name"]
    export["type"] = poly["type"]

    output = open(result_file, "w")
    output.write(json.dumps(export))
    output.close()
    
    if instance_name == "ccheese142.instance.json":
        render_polygon(poly, solution=export)

print(f"Total score: {total_score}")

ccheese142.instance.json (rotated 1 times, seed 0) - Faces: 96
ccheese4390.instance.json (rotated 3 times, seed 0) - Faces: 3224
example_instance1.instance.json (rotated 0 times, seed 0) - Faces: 5
fpg-poly_0000000020_h1.instance.json (rotated 1 times, seed 0) - Faces: 11
fpg-poly_0000000020_h2.instance.json (rotated 0 times, seed 0) - Faces: 17
fpg-poly_0000004900_h2.instance.json (rotated 2 times, seed 0) - Faces: 2492
maze_4344_250_001_01.instance.json (rotated 1 times, seed 0) - Faces: 1286
maze_79_50_05_005.instance.json (rotated 3 times, seed 0) - Faces: 29
socg60.instance.json (rotated 0 times, seed 0) - Faces: 15
srpg_iso_aligned_mc0000088.instance.json (rotated 1 times, seed 0) - Faces: 26
srpg_iso_aligned_mc0001336.instance.json (rotated 1 times, seed 0) - Faces: 450
srpg_iso_mc0000080.instance.json (rotated 2 times, seed 0) - Faces: 41
srpg_octa_mc0000082.instance.json (rotated 0 times, seed 0) - Faces: 28
srpg_octa_mc0000784.instance.json (rotated 1 times, seed 0) - Faces: 